In [22]:
import mlflow
from mlflow.tracking import MlflowClient
import pandas as pd

In [2]:
MLFLOW_TRACKING_URI = 'sqlite:///cred_sqlite_mlflow.db'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [3]:
experiments = client.search_experiments()

for exp in experiments:
    print(f"Name: {exp.name}, ID: {exp.experiment_id}, Artifact Location: {exp.artifact_location}")

Name: credit_default_risk_experiment_tracking, ID: 1, Artifact Location: /Users/mac/Projects/MLops_datatalks_project/02-experiment-tracking/mlruns/1
Name: Default, ID: 0, Artifact Location: mlflow-artifacts:/0


In [4]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids = '1',
    filter_string = 'metrics.auc > 0.733',
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 5,
    order_by = ['metrics.auc ASC']
)

In [7]:
for run in runs:
    print(f"runId: {run.info.run_id}, auc: {run.data.metrics['auc']:.4f}")

runId: 2c2f5792316545ed84ddf88b09b072a9, auc: 0.7340
runId: f7221513467b4f8ea15d8ca3054c8224, auc: 0.7340
runId: c52ab90a9f3241f09a8b9db4829140e4, auc: 0.7340


In [8]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [9]:
run_id = 'c52ab90a9f3241f09a8b9db4829140e4'
model_uri = f'runs:/{run_id}/models/xgboost_model'
mlflow.register_model(model_uri=model_uri,name='credit_default_risk_xgb_model_v2')

Successfully registered model 'credit_default_risk_xgb_model_v2'.
Created version '1' of model 'credit_default_risk_xgb_model_v2'.


<ModelVersion: aliases=[], creation_timestamp=1754337563297, current_stage='None', description=None, last_updated_timestamp=1754337563297, name='credit_default_risk_xgb_model_v2', run_id='c52ab90a9f3241f09a8b9db4829140e4', run_link=None, source='/Users/mac/Projects/MLops_datatalks_project/02-experiment-tracking/mlruns/1/c52ab90a9f3241f09a8b9db4829140e4/artifacts/models/xgboost_model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [19]:
model_name = 'credit_default_risk_xgb_model_v1'
latest_versions = client.get_latest_versions(name=model_name)
    
for version in latest_versions:
    print(f'version: {version.version}, stage: {version.current_stage}')

version: 1, stage: None
version: 2, stage: Production


/var/folders/gy/0jy_9kfn7c5b6pxfsksq3m8w0000gn/T/ipykernel_30729/3326168028.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [15]:
model_version = 2
new_stage = 'Staging'

In [17]:
model_version = 2
new_stage = 'Production'

In [18]:
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/var/folders/gy/0jy_9kfn7c5b6pxfsksq3m8w0000gn/T/ipykernel_30729/227647614.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1754335560720, current_stage='Production', description='', last_updated_timestamp=1754337908831, name='credit_default_risk_xgb_model_v1', run_id='2c2f5792316545ed84ddf88b09b072a9', run_link='', source='models:/credit_default_risk_xgb_model_v1/1', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [20]:
from datetime import datetime as dt

In [21]:
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f'model version {model_version} was transitioned to {new_stage} on {dt.today().now() }'
)

<ModelVersion: aliases=[], creation_timestamp=1754335560720, current_stage='Production', description='model version 2 was transitioned to Production on 2025-08-04 21:05:31.866639', last_updated_timestamp=1754337931867, name='credit_default_risk_xgb_model_v1', run_id='2c2f5792316545ed84ddf88b09b072a9', run_link='', source='models:/credit_default_risk_xgb_model_v1/1', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [23]:
test_df = pd.read_parquet('../processed_data/X_test.parquet')

In [25]:
test_df.head()

,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,FLOORSMAX_AVG,AGE_GROUP,YEARS_EMPLOYED_GROUP,PHONE_CHANGE_GROUP
17424,3,3,0.441836,0.286877,0.559264,0.1667,Youth,1-5 yrs,Recent
305296,2,2,0.472253,0.705166,0.456198,0.1667,Youth,1-5 yrs,Old
17224,2,2,0.684828,0.590742,0.626568,0.0417,Youth,1-5 yrs,Moderate
215169,2,2,0.574447,0.092250,0.505998,0.1667,Middle_Age,unemployed and <1 yr,Moderate
181299,2,2,0.417100,0.631203,0.505998,0.0000,Middle_Age,1-5 yrs,Moderate


In [33]:
client.download_artifacts(run_id=run_id, path='', dst_path='.')

'/Users/mac/Projects/MLops_datatalks_project/02-experiment-tracking/'

In [ ]:
import pickle

with open('models/model.xgb', 'rb') as f_in:
    dv = pickle.load(f_in)